In [1]:
#
# Only run this code if you want to see lots of debug!!
#
#import logging

#handler = logging.StreamHandler()
#for l in ['ncclient.transport.session', 'ncclient.operations.rpc']:
#    logger = logging.getLogger(l)
#    logger.addHandler(handler)
#    logger.setLevel(logging.DEBUG)

Put in your own details below...this IP address is a test machine for the DMI team in SJC.

In [2]:
# ASR1K
HOST = '172.27.255.40'
PORT = 830
USER = 'admin'
PASS = 'admin'

In [1]:
# 3850
HOST = '10.85.70.86'
PORT = 830
USER = 'admin'
PASS = 'admin'

### Basic YDK initialization

In [2]:
from ydk.types import Empty
from ydk.errors import YPYError
from ydk.services import CRUDService
from ydk.providers import NetconfServiceProvider

session = NetconfServiceProvider(
    address=HOST,
    port=PORT,
    username=USER,
    password=PASS,
    protocol='ssh')

# Initialize a CRUD service
c = CRUDService()

SSHError: Could not open socket to 10.85.70.86:830

### Some helpers to print, create and delete Loopbacks

In [4]:
import ydk.models.ned as ned

# create a Loopback
def create_loopback(index):
    new_loopback = ned.Native.Interface.Loopback()
    new_loopback.name = index
    c.create(session, new_loopback)

# create a list of Loopbacks
def create_loopback_list(index_list):
    intf_container = ned.Native.Interface()
    for index in index_list:
        new_loopback = ned.Native.Interface.Loopback()
        new_loopback.name = index
        new_loopback.description = "Created Interface {}".format(index)
        intf_container.loopback.append(new_loopback)
    c.create(session, intf_container)

# delete a Loopback
def delete_loopback(index):
    new_loopback = ned.Native.Interface.Loopback()
    new_loopback.name = index
    c.delete(session, new_loopback)

def print_loopbacks():
    q_loopbacks = ned.Native.Interface.Loopback()
    loopbacks = c.read(session, q_loopbacks)
    print "Loopbacks:"
    for l in loopbacks:
        print "  {} {}".format(l.name, l.description)

In [5]:
create_loopback(123)
print_loopbacks()
delete_loopback(123)
print_loopbacks()

Loopbacks:
  0 None
  1 None
  2 None
  123 None
  9999 LB 9999
Loopbacks:
  0 None
  1 None
  2 None
  9999 LB 9999


In [6]:
create_loopback_list([5,6,7])
print_loopbacks()

Loopbacks:
  0 None
  1 None
  2 None
  5 Created Interface 5
  6 Created Interface 6
  7 Created Interface 7
  9999 LB 9999


In [7]:
delete_loopback(5)
delete_loopback(6)
delete_loopback(7)
print_loopbacks()

Loopbacks:
  0 None
  1 None
  2 None
  9999 LB 9999


### Some helpers to print, create and delete Loopbacks

In [8]:
import ydk.models.ned as ned

def print_vrfs():
    q_vrf = ned.Native.Vrf()
    vrfs = c.read(session, q_vrf)
    print("VRFs:")
    for v in vrfs.definition:
        print("  {}".format(v.name))

# add a new VRF
def add_vrf(name, rd):
    vrfs = ned.Native.Vrf()
    new_vrf = ned.Native.Vrf.Definition()
    new_vrf.name = name
    new_vrf.rd = rd
    vrfs.definition.append(new_vrf)
    c.create(session, vrfs)

# delete a VRF
def delete_vrf(name):
    to_delete = ned.Native.Vrf.Definition()
    to_delete.name = name
    c.delete(session, to_delete)

In [9]:
print_vrfs()

VRFs:
  Mgmt-intf
  mpls-vrf


In [10]:
add_vrf('foo', '1:1')
add_vrf('bar', '1:1')

print_vrfs()

VRFs:
  Mgmt-intf
  bar
  foo
  mpls-vrf


In [11]:
delete_vrf('foo')
delete_vrf('bar')

print_vrfs()

VRFs:
  Mgmt-intf
  mpls-vrf
